# En este colab se prueba el modelo entrenado por nosotros para textual entailment sobre XNLI (mismo proceso que el hecho por BSC para generar el BNE-TE), más un finetunning adicional sobre frases de saludos y ayuda.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MAF-ProyectoNLP/Validacion-Modelos/blob/main/Entrenamiento%20propio%20%2B%20Finentuning%20Saludos%20-%20validation%20set%20conversación.ipynb)


*0*. Se instalan las bibliotecas necesarias

In [ ]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 947.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 11.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 82.7 MB/s eta 0:00

*1*. Se importan las dependencias

In [ ]:
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import datasets
import numpy as np
import random
import pandas as pd

from transformers import  RobertaForSequenceClassification, RobertaConfig, RobertaTokenizer

from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

*2*. Se descarga el modelo fintuneado con frases de saludo y ayuda y se define el tipo de modelo. También el dataset a utilizar para evaluar, en este caso el validation set de conversaciones.

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

!mkdir "kfold"
file_id = '1ZlFvCmncexUCWTeK_Ellr9V-NjOLj4m1' 
download = drive.CreateFile({'id': file_id})
download.GetContentFile('kfold/config.json')
file_id ='1-4BA1rCp5HhShofsbA9-cOLDBD6Z9XLl'
download = drive.CreateFile({'id': file_id})
download.GetContentFile('kfold/pytorch_model.bin')

In [ ]:
MODEL_TYPE = "PlanTL-GOB-ES/roberta-large-bne-te"

In [ ]:
MAX_LEN=256
BATCH_SIZE = 8
NUM_CORES=1

In [ ]:
dataset_conversaciones='/content/Entrenamiento-y-Validacion/dataset_propio_validation_set.csv'

*3*. TestDataset and CompDataset son las clases usadas para la tokenización de la frases, CompDataset se utiliza para entrenar (devuelve también el label), y TestDataset es utilizada para tokenizar las frases para validación

In [ ]:
class TestDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,
                    add_special_tokens = True, 
                    max_length = MAX_LEN, 
                    pad_to_max_length = True,
                    return_attention_mask = True,  
                    return_tensors = 'pt',    
                    padding="max_length", 
                    truncation=True
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]

        sample = (padded_token_list, att_mask)
        return sample

    def __len__(self):
        return len(self.df_data)

In [ ]:
class CompDataset(Dataset):

    def __init__(self, df):
        self.df_data = df

    def __getitem__(self, index):
        sentence1 = self.df_data.loc[index, 'premise']
        sentence2 = self.df_data.loc[index, 'hypothesis']

        encoded_dict = tokenizer.encode_plus(
                    sentence1, sentence2,   
                    add_special_tokens = True,  
                    max_length = MAX_LEN,        
                    pad_to_max_length = True,
                    return_attention_mask = True,
                    return_tensors = 'pt', 
                    padding="max_length", 
                    truncation=True
               )
        
        padded_token_list = encoded_dict['input_ids'][0]
        att_mask = encoded_dict['attention_mask'][0]
        
        target = torch.tensor(self.df_data.loc[index, 'label'])

        sample = (padded_token_list, att_mask, target)
        return sample

    def __len__(self):
        return len(self.df_data) 

*4*. Se carga el modelo

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained(MODEL_TYPE)

In [ ]:
model_parameters =  RobertaForSequenceClassification.from_pretrained(MODEL_TYPE, num_labels=3)
config = RobertaConfig(
                vocab_size= model_parameters.config.vocab_size,
                hidden_size= model_parameters.config.hidden_size,
                num_hidden_layers= model_parameters.config.num_hidden_layers,
                num_attention_heads=model_parameters.config.num_attention_heads,
                intermediate_size=model_parameters.config.intermediate_size,
                hidden_act=model_parameters.config.hidden_act,
                hidden_dropout_prob=model_parameters.config.hidden_dropout_prob,
                attention_probs_dropout_prob=model_parameters.config.attention_probs_dropout_prob,
                max_position_embeddings=model_parameters.config.max_position_embeddings,
                type_vocab_size=model_parameters.config.type_vocab_size,
                initializer_range=model_parameters.config.initializer_range,
            )

config.num_labels = 3

print(config)

RobertaConfig {
  "attention_probs_dropout_prob": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.4",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50262
}



In [ ]:
model = RobertaForSequenceClassification(config)
model = RobertaForSequenceClassification.from_pretrained('/content/kfold/')

In [ ]:
print(model.roberta.embeddings.word_embeddings.weight.shape)

torch.Size([50262, 1024])


In [ ]:
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50262, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

*5*. Se define un conjunto de frases para validar el algoritmo sobre conversaciones de prueba

In [ ]:
!git clone "https://github.com/MAF-ProyectoNLP/Entrenamiento-y-Validacion"

Cloning into 'Entrenamiento-y-Validacion'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (4/4), 3.12 KiB | 3.12 MiB/s, done.


*6*. Métricas

*6.1* Se crea la función metrics_calculation para evaluar las métricas generales del modelo sobre un dataset

In [ ]:
def metrics_calculation():
  frases=pd.read_csv(dataset_conversaciones, encoding='Windows-1252', delimiter=",")
  frases=frases[['premise','hypothesis','label']]

  # Set the seed.
  seed_val = 101

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  loss_values = []

  dataset = CompDataset(frases)

  val_dataloader = torch.utils.data.DataLoader(dataset,
                    batch_size=BATCH_SIZE,shuffle=False,num_workers=NUM_CORES)

  model.eval()

  torch.set_grad_enabled(False)

  total_val_loss = 0
  stacked_val_labels = []
  targets_list = []
  for j, batch in enumerate(val_dataloader):
      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)      

      outputs = model(b_input_ids, 
              attention_mask=b_input_mask, 
              labels=b_labels)
      
      loss = outputs[0]
      total_val_loss = total_val_loss + loss.item()

      preds = outputs[1]
      val_preds = preds.detach().cpu().numpy()
      targets_np = b_labels.to('cpu').numpy()

      targets_list.extend(targets_np)

      if j == 0:  # first batch
          stacked_val_preds = val_preds
      else:
          stacked_val_preds = np.vstack((stacked_val_preds, val_preds))



  y_true = targets_list
  y_pred = np.argmax(stacked_val_preds, axis=1)

  y_true = np.array(y_true)  

  y_pred[y_pred==2] = 1   

  val_acc = accuracy_score(y_true, y_pred)
  prec_sc = precision_score(y_true, y_pred)
  rec_sc = recall_score(y_true, y_pred)
  f1_sc = f1_score(y_true, y_pred)


  #print('Val loss:' ,total_val_loss)
  print('Accuracy: ', val_acc)
  print('Precision: ', prec_sc)
  print('Recall: ', rec_sc)
  print('F1: ', f1_sc)

*6.2* Se define una métrica y las estructuras para evaluar las conversaciones, y se crea una matriz de predicciones para poder realizar cálculos sobre la misma

In [ ]:
def init_stats(hypotesis_to_be_tested):
   categories=['Presente','OrdenDistinto','NoPresente']
   global scores
   scores= pd.DataFrame((np.outer(np.zeros(len(hypotesis_to_be_tested)),np.zeros(len(categories)))),columns=categories, index=hypotesis_to_be_tested)
   
   global truth_scores
   truth_scores= pd.DataFrame((np.outer(np.zeros(len(hypotesis_to_be_tested)),np.zeros(len(categories)))),columns=categories, index=hypotesis_to_be_tested)

 > getTruthLabels calcula la salida real

In [ ]:
def getTruthLabels(hyps,conversation):
  text="Salida esperada según estandar: "
  M = np.zeros([len(hyps),len(conversation)])
  
  frases=pd.read_csv(dataset_conversaciones, encoding='Windows-1252', delimiter=",")
  frases=frases[['premise','hypothesis','label']]
  for h in range(len(hyps)):
    for c in range(len(conversation)):
      if conversation[c] in frases['premise'].tolist():
        res=frases[(frases['premise']==conversation[c])]
        res=res[res['hypothesis']==hyps[h]]['label'].iloc[0]
        M[h,c]=res
      else:
        M[h,c]=-1

  #print(M)
  for hyp in range(len(M)):
    h_conv=M[hyp]
    if (h_conv[hyp]==0):
      text += str(hyps[hyp] + " presente. ")
      truth_scores.loc[hyps[hyp]]['Presente']+=1
    else:
      entailment=False
      for i in range(len(h_conv)):
        if h_conv[i]==0:
          text += str(hyps[hyp] +" desordenado. ")
          truth_scores.loc[hyps[hyp]]['OrdenDistinto']+=1
          entailment=True
          break
      if i+1==len(h_conv) and not entailment:
          text += str(hyps[hyp] +" no presente.")
          truth_scores.loc[hyps[hyp]]['NoPresente']+=1

  return text

> calcular_diferencia evalúa la diferencia entre el entailment de las frases hecho por el algoritmo y lo teórico

In [ ]:
def calcular_diferencia():
  scores['Total']=scores['Presente']+scores['OrdenDistinto']+scores['NoPresente']
  truth_scores['Total']=truth_scores['Presente']+truth_scores['OrdenDistinto']+truth_scores['NoPresente']
  if truth_scores['Total'][0]==scores['Total'][0]:
    res=scores-truth_scores
    res.rename(columns = {'Presente':'Diferencia Hipotesis Presentes','OrdenDistinto':'Diferencia Orden Hipotesis','NoPresente':'Diferencia Hipotesis No Presentes','Total':'Premisas Totales'}, inplace = True)
    res['Premisas Totales']=scores['Total']
    return res
  else:
    print("La cantidad de predicciones no coinciden")
    return None

La función getlabel predice el entailment de una frase, que la recibe como una TestDataset 

In [ ]:
def get_label(test_dataloader):
  total = 0

  for _, batch in enumerate(test_dataloader):

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)  

    import os
    os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
    outputs = model(b_input_ids, attention_mask=b_input_mask)
    preds = outputs[0].detach().cpu().numpy()
    y_pred = np.argmax(preds, axis=1)
  
  return y_pred

 > predict_matrix calcula una matriz de entailments, indicando en la posición (i,j) el entailment de la hipotesis i sobre la premisa j

In [ ]:
def predict_matrix(hypotesis_to_be_tested,answers_from_operator):
  M = np.zeros([len(hypotesis_to_be_tested),len(answers_from_operator)])
  
  for i in range(len(hypotesis_to_be_tested)):
      for j in range(len(answers_from_operator)):
        dataset = {'premise': [answers_from_operator[j]], 'hypothesis': [hypotesis_to_be_tested[i]]}

        dataloader = torch.utils.data.DataLoader(TestDataset(pd.DataFrame(dataset)), batch_size=1, shuffle=False, num_workers=1)
        M[i,j] = get_label(dataloader)
        #print(str(i)+ " hyp "+ hypotesis_to_be_tested[i]+ " | "+str(j)+ " prem "+ answers_from_operator[j]+ " : "+ str(M[i,j]))
  return(M)

*7*. Se crea la función conversation_analysis, que recibe como parámetros una lista de hipótesis a ser testeadas, y las respuestas brindadas por la operadora. Se utiliza como base la función predict_matrix para indicar para cada hipótesis si se encuentra presente en las respuestas, y en el órden correcto. 



In [ ]:
def conversation_analysis(hypotesis_to_be_tested,answers_from_operator):
  out=predict_matrix(hypotesis_to_be_tested,answers_from_operator)
  #print(out)
  hypotesis_ordered= np.ones([len(hypotesis_to_be_tested)])*-1
  for h in range(len(hypotesis_to_be_tested)):
    checker= np.zeros([len(hypotesis_to_be_tested)])
    checker[h]=1
    c=out.T.dot(checker)
    # hace entailment para la hipotesis en el orden correcto 
    if c[h]==0:
      hypotesis_ordered[h]=h
    # busco la primer ubicación dónde se hace entailment
    else:
      for x in range(len(c)):
        if c[x]==0:
          hypotesis_ordered[h]=x
          break

  # en caso de que 2 premisas cumplan 2 hipótesis, verifico si hay una combinación que haga mayor cantidad de entailments  
  for x in range(1,len(hypotesis_ordered)):
    if hypotesis_ordered[x-1]== hypotesis_ordered[x]:
        if out[x,x]>0 and out[x-1,x]==0 and out[x-1,x-1]==0:
          hypotesis_ordered[x-1]=x
        elif out[x-1,x]>0 and out[x,x]==0 and out[x-1,x-1]==0:
          hypotesis_ordered[x]=x
        elif out[x-1,x]>0 and out[x,x]==0 and out[x-1,x-1]==0:
          hypotesis_ordered[x]=x
          

  ok=[]
  wrong=[]

  #print("Array de hipotesis ordenadas:")
  #print(hypotesis_ordered)
  for i in range(len(hypotesis_ordered)):
    if int(hypotesis_ordered[i]) ==i:
      ok.append(hypotesis_to_be_tested[i] +" correctamente presente en la comunicación")
      scores.loc[hypotesis_to_be_tested[i]]['Presente']+=1
      
    elif int(hypotesis_ordered[i])<0:
      wrong.append(hypotesis_to_be_tested[i] +" NO PRESENTE en la comunicación")
      scores.loc[hypotesis_to_be_tested[i]]['NoPresente']+=1   

    else:
      val=int(hypotesis_ordered[i])
      if int(hypotesis_ordered[val])==val:
        wrong.append(hypotesis_to_be_tested[i] +" NO PRESENTE en la comunicación")
        scores.loc[hypotesis_to_be_tested[i]]['NoPresente']+=1  
      else:
        wrong.append(hypotesis_to_be_tested[i] +" presente en la comunicación, pero NO EN EL ORDEN CORRECTO")
        scores.loc[hypotesis_to_be_tested[i]]['OrdenDistinto']+=1  

  print("** Devolución de la comunicación realizado por la operadora **")
  if len(wrong)>0:
    print("A mejorar:")
  else:
    ok.append("Felicitaciones! Comunicación realizada de forma deseada")
  for w in wrong:
    print(w)

  if len(ok)>0:
    print("Positivo:")
  for o in ok:
    print(o)

#  print(getTruthLabels(hypotesis_to_be_tested,answers_from_operator))

*8*. **PRUEBA**

*8.1* La lista de hipótesis a testear

In [ ]:
hypotesis_to_be_tested=["Saludo","Ayuda"]

In [ ]:
init_stats(hypotesis_to_be_tested)

In [ ]:
def get_conversation_analysis(answers_from_operator):
  conversation_analysis(hypotesis_to_be_tested,answers_from_operator)

*8.2* Se testean casos distintos de respuestas, a fin de evaluar el modelo

In [ ]:
get_conversation_analysis(["Hola, ¿cómo estás?", "¿Puedo ayudarte con algo?"])

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ayuda correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [ ]:
get_conversation_analysis(["Buen día, ¿cómo te va?", "Si necesitas cualquier cosa, no dudes en pedírmelo."])

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ayuda correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [ ]:
get_conversation_analysis(["Buenos días, ¿cómo amaneciste?", "¿Quieres que te eche una mano?"])

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ayuda correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [ ]:
get_conversation_analysis(["Hola, ¿qué tal estás?", "¿Has leído el último libro de Stephen King?"])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ayuda NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


In [ ]:
get_conversation_analysis(["Buenas tardes, ¿cómo ha ido tu día?", "¿Te gustaría que te acompañe al médico?"])

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ayuda correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [ ]:
get_conversation_analysis(["Hola, ¿qué hay de nuevo?", "Si necesitas alguien que te ayude a mover muebles, estaré encantado/a de hacerlo."])

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ayuda correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [ ]:
get_conversation_analysis(["¿Qué tal todo?", "Me gustaría visitar algún día la ciudad de Nueva York."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ayuda NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


In [ ]:
get_conversation_analysis(["Hola, ¿cómo te sientes hoy?", "¿Quieres que te lleve a la estación de tren?"])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ayuda NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


In [ ]:
get_conversation_analysis(["Buen día, ¿cómo ha estado tu semana?", "¿Necesitas ayuda con la mudanza?"])

** Devolución de la comunicación realizado por la operadora **
Positivo:
Saludo correctamente presente en la comunicación
Ayuda correctamente presente en la comunicación
Felicitaciones! Comunicación realizada de forma deseada


In [ ]:
get_conversation_analysis(["Hola, ¿cómo te ha ido últimamente?", "Estoy pensando en comprarme un nuevo teléfono móvil."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Ayuda NO PRESENTE en la comunicación
Positivo:
Saludo correctamente presente en la comunicación


In [ ]:
get_conversation_analysis(["El cielo está muy despejado hoy.", "Me gusta mucho el color de tu camisa."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["¿Has visto la última película de terror que salió?", "No puedo creer que haya tanto tráfico hoy."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["¿Qué opinas del cambio climático?", "Si necesitas una persona que te escuche, puedes contar conmigo."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["Tengo mucho trabajo que hacer hoy.", "Me encanta viajar y conocer nuevas culturas."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["No puedo creer que ya sea marzo.", "¿Ya has ido a ese nuevo restaurante de comida mexicana?"])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["¿Ya hiciste tus compras de la semana?", "¿Te gusta la música clásica?"])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["Me encanta leer novelas de misterio.", "¿Quieres que te ayude a preparar la cena?"])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Positivo:
Ayuda correctamente presente en la comunicación


In [ ]:
get_conversation_analysis(["¿Te gustan los animales exóticos?", "¿Tienes planes para el fin de semana?"])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["La verdad es que estoy muy cansado/a.", "La verdad es que no me gusta mucho el invierno."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


In [ ]:
get_conversation_analysis(["Mi deporte favorito es el fútbol.", "Si necesitas cualquier cosa durante el evento, por favor, házmelo saber."])

** Devolución de la comunicación realizado por la operadora **
A mejorar:
Saludo NO PRESENTE en la comunicación
Ayuda NO PRESENTE en la comunicación


A nivel de caso de negocio, se imprime las predicciones, para ver en qué están fallando en el flujo de la comunicación.

In [ ]:
scores

,Presente,OrdenDistinto,NoPresente
Saludo,10.0,0.0,10.0
Ayuda,7.0,0.0,13.0


Por último, se imprimen las métricas definidas para evaluar y comparar los modelos

In [ ]:
metrics_calculation()

Accuracy:  0.9459459459459459
Precision:  0.9157894736842105
Recall:  1.0
F1:  0.956043956043956
